In [2]:
from typing import Dict
import sqlite3
from redis import Redis
from tqdm import tqdm
import json
from discord import Message

import config
from parrot.redis.avatar_manager import AvatarManager as RedisAvatarManager
from parrot.redis.corpus_manager import CorpusManager as RedisCorpusManager
from parrot.redis.redis_set import RedisSet
from parrot.sqlite.corpus_manager import CorpusManager as SqliteCorpusManager


In [3]:
print("Logging into the Redis database...")
redis = Redis(
    host=config.REDIS_HOST,
    port=config.REDIS_PORT,
    password=config.REDIS_PASSWORD,
    decode_responses=True,
)

print("Connecting to the sqlite database...")
con = sqlite3.connect("parrot.sqlite3")
# con.isolation_level = None  # autocommit mode

Logging into the Redis database...
Connecting to the sqlite database...


In [4]:
print("Instantiating data structures...")
redis_corpora = RedisCorpusManager(redis)
sqlite_corpora = SqliteCorpusManager(con)
redis_avatars = RedisAvatarManager(redis)
redis_registered_users = RedisSet(redis, "registered_users")
redis_learning_channels = RedisSet(redis, "learning_channels")
redis_speaking_channels = RedisSet(redis, "speaking_channels")

Instantiating data structures...


In [ ]:
print("Creating tables if they don't exist...")
con.executescript("""
    BEGIN;
    CREATE TABLE IF NOT EXISTS users (
        id                         INTEGER PRIMARY KEY,
        is_registered              INTEGER NOT NULL DEFAULT 0,
        original_avatar_url        TEXT,
        modified_avatar_url        TEXT,
        modified_avatar_message_id INTEGER
    );

    CREATE TABLE IF NOT EXISTS channels (
        id             INTEGER PRIMARY KEY,
        can_speak_here INTEGER NOT NULL DEFAULT 0,
        can_learn_here INTEGER NOT NULL DEFAULT 0,
        webhook_id     INTEGER
    );

    CREATE TABLE IF NOT EXISTS messages (
        id        INTEGER PRIMARY KEY,
        user_id   INTEGER NOT NULL REFERENCES users(id),
        timestamp INTEGER NOT NULL,
        content   TEXT    NOT NULL
    );
    COMMIT;
""")

In [ ]:
for user_id in tqdm(redis_registered_users, "User registration"):
    con.execute("INSERT INTO users (id, is_registered) VALUES (?, ?)", (user_id, 1))
con.commit()

In [5]:
print("Collecting corpus keys...", end="")
corpus_keys = redis.keys("corpus:*")
print(f" ✅ {len(corpus_keys)} keys")

In [6]:
for key in tqdm(corpus_keys, "Corpora"):
    user_id = int(key.split(":")[1])
    corpus: Dict[int, str] = redis.hgetall(key)
    # Timestamp was not stored in the old database, but I want it in the new one
    # so I'm just going to put 0 for now.
    con.executemany("""
        INSERT OR IGNORE INTO messages (id, user_id, timestamp, content)
        VALUES (?, ?, ?, ?)""",
        [(int(message_id), user_id, 0, corpus[message_id]) for message_id in corpus]
    )
con.commit()

Corpora: 100%|██████████| 227/227 [02:04<00:00,  1.83it/s]


In [9]:
for user_id, ledger in tqdm(redis.hgetall("avatars").items(), "Avatars"):
    user_id = int(user_id)
    ledger = json.loads(ledger)
    sql = """
        UPDATE users
        SET original_avatar_url = ?,
            modified_avatar_url = ?,
            modified_avatar_message_id = ?
        WHERE id = ?
    """
    con.execute(sql, (
        ledger["original_avatar_url"],
        ledger["modified_avatar_url"],
        ledger["source_message_id"],
        user_id
    ))
con.commit()

Avatars: 100%|██████████| 151/151 [00:00<00:00, 43178.34it/s]


In [12]:
for channel_id in tqdm(redis_speaking_channels, "Speaking permissions"):
    con.execute("""
        INSERT INTO channels (id, can_speak_here)
        VALUES (?, ?)
        ON CONFLICT (id) DO UPDATE
        SET can_speak_here = EXCLUDED.can_speak_here""",
        (channel_id, 1)
    )
con.commit()

Speaking permissions: 100%|██████████| 12/12 [00:00<00:00, 3000.93it/s]


In [13]:
for channel_id in tqdm(redis_learning_channels, "Learning permissions"):
    con.execute("""
        INSERT INTO channels (id, can_learn_here)
        VALUES (?, ?)
        ON CONFLICT (id) DO UPDATE
        SET can_speak_here = EXCLUDED.can_learn_here""",
        (channel_id, 1)
)
con.commit()

Learning permissions: 100%|██████████| 4/4 [00:00<00:00, 3997.43it/s]


In [ ]:
# webhooks are funny, because parrot doesn't actually store them; it just gets
# each url on the fly by searching through the channel's list of webhooks on
# discord. might be easier to poke at parrot while its live than to bring in all
# the discord bot code here just to get at this data.

# for channel_id in tqdm(redis_speaking_channels, "Webhooks"):
#     ...
# con.commit()

In [14]:
con.commit()
con.close()